In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import joblib

In [3]:
df_clean = pd.read_csv("../multi_class_grid_clean.csv")

In [4]:
print(df_clean.select_dtypes(include=["object"]).columns.tolist())

['Polo']


In [5]:
print(df_clean["Polo"].unique())

['Porto Alegre' 'Marabá' 'Brasília' 'Belo Horizonte' 'Juazeiro do Norte'
 'Recife']


In [6]:
polo_test_value = "Belo Horizonte" 

In [7]:
train_df = df_clean[df_clean["Polo"] != polo_test_value].copy()
test_df  = df_clean[df_clean["Polo"] == polo_test_value].copy()

In [9]:
X_train = train_df.drop(columns=["target", "Polo"])
y_train = train_df["target"]

X_test = test_df.drop(columns=["target", "Polo"])
y_test = test_df["target"]

In [10]:
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_main)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

best_auc = 0
best_C = None

for C in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(C=C, max_iter=1000, random_state=42)
    model.fit(X_train_main, y_train_main)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_val_proba)
    
    print(f"C={C}, Validation AUC={auc:.3f}")
    
    if auc > best_auc:
        best_auc = auc
        best_C = C

print(f"Best C: {best_C} with AUC={best_auc:.3f}")

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C=0.01, Validation AUC=0.738


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C=0.1, Validation AUC=0.737


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C=1, Validation AUC=0.738


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C=10, Validation AUC=0.737
C=100, Validation AUC=0.737
Best C: 1 with AUC=0.738


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
logreg = LogisticRegression(C=best_C, max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train_main)


LogisticRegression(C=1, max_iter=1000, random_state=42)

In [14]:
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

# Probabilities of class 1 (positive)
y_proba = logreg.predict_proba(X_test_scaled)[:, 1]

# Predicted classes (0 or 1)
y_pred = logreg.predict(X_test_scaled)

# AUC
auc = roc_auc_score(y_test, y_proba)
print(f"AUC: {auc:.3f}")

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Detailed classification report
print(classification_report(y_test, y_pred))

AUC: 0.748
Accuracy: 0.781
Confusion Matrix:
 [[139732     47]
 [ 39316    686]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.88    139779
           1       0.94      0.02      0.03     40002

    accuracy                           0.78    179781
   macro avg       0.86      0.51      0.46    179781
weighted avg       0.82      0.78      0.69    179781

